In [2]:
import pandas as pd
import numpy as np

school_data_to_load = "schools_complete.csv"
student_data_to_load = "students_complete.csv"

school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"]) 

In [3]:
school_data = school_data.rename(columns={"name":"SchoolName"})
student_data = student_data.rename(columns={"school":"SchoolName"})

In [4]:
unique_schools = school_data_complete['school_name'].unique()
total_schools = len(unique_schools)

total_students = school_data_complete['student_name'].count()

unique_budget = school_data_complete['budget'].unique()
total_budget = sum(unique_budget)

avg_math_score = school_data_complete['math_score'].mean()

avg_reading_score = school_data_complete['reading_score'].mean()

pass_math = school_data_complete.loc[school_data_complete['math_score'] >= 70]['math_score'].count()
perc_pass_math = pass_math/total_students

pass_reading = school_data_complete.loc[school_data_complete['reading_score'] >= 70]['reading_score'].count()
perc_pass_reading = pass_reading/total_students

overall_pass = (perc_pass_math + perc_pass_reading)/2

district_summary = pd.DataFrame({
    "Total Schools": [total_schools],
    "Total Budget": [total_budget],
    "Average Math Score": [avg_math_score],
    "Average Reading Score": [avg_reading_score],
    "% Passing Math": [perc_pass_math],
    "% Passing Reading": [perc_pass_reading],
    "% Overall Passing Rate": [overall_pass]
})

district_summary.style.format({"Total Budget": "${:,.2f}", 
                       "Average Reading Score": "{:.1f}", 
                       "Average Math Score": "{:.1f}", 
                       "% Passing Math": "{:.1%}", 
                       "% Passing Reading": "{:.1%}", 
                       "% Overall Passing Rate": "{:.1%}"})

,Total Schools,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"$24,649,428.00",79.0,81.9,75.0%,85.8%,80.4%


In [6]:
group_by_school = student_data.groupby(["school_name"], as_index=False)

avg_math = pd.DataFrame(group_by_school["math_score"].mean())

avg_reading = pd.DataFrame(group_by_school["reading_score"].mean())

group_pass_math = student_data[student_data["math_score"]>70].groupby(["school_name"], as_index=False)

pass_math = pd.DataFrame(group_pass_math["math_score"].count())
pass_math.columns= ["school_name", "math_percentage"]

group_pass_reading = student_data[student_data["reading_score"]>70].groupby(["school_name"], as_index=False)

pass_reading = pd.DataFrame(group_pass_reading["reading_score"].count())
pass_reading.columns= ["school_name", "reading_percentage"]

merged_data = pd.merge(school_data, avg_math, on="school_name" )
merged_data = pd.merge(merged_data, pass_math, on="school_name" )
merged_data = pd.merge(merged_data, avg_reading, on="school_name" )
merged_data = pd.merge(merged_data, pass_reading, on="school_name" )

merged_data["math_percentage"] = (merged_data["math_percentage"]/merged_data["size"])*100
merged_data["reading_percentage"] = (merged_data["reading_percentage"]/merged_data["size"])*100

merged_data["overall_percentage"] = (merged_data["math_percentage"]+merged_data["reading_percentage"])/2

merged_data["perStudentBudget"] = merged_data["budget"]/merged_data["size"]

school_summary_og = pd.DataFrame(merged_data[["school_name", "type", "size", "budget", "perStudentBudget", "math_score", "reading_score", "math_percentage", "reading_percentage", "overall_percentage"]])
school_summary_og.columns = ["School Name", "Type", "Total Students", "Budget", "Per Student Budget", "Avg Math", "Avg Reading", "% Passing Math", "% Passing Reading", "% Overall Pass Rate"]

school_summary = pd.DataFrame(merged_data[["school_name", "type", "size", "budget", "perStudentBudget", "math_score", "reading_score", "math_percentage", "reading_percentage", "overall_percentage"]])
school_summary.columns = ["School Name", "Type", "Total Students", "Budget", "Per Student Budget", "Avg Math", "Avg Reading", "% Passing Math", "% Passing Reading", "% Overall Pass Rate"]

school_summary["Num Students"] = school_summary["Total Students"].map("{:,}".format)
school_summary["Budget"] = school_summary["Budget"].map("${:,}".format)
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].map("${:,}".format)
school_summary["Avg Math"] = school_summary["Avg Math"].map("{:.2f}%".format)
school_summary["Avg Reading"] = school_summary["Avg Reading"].map("{:.2f}%".format)
school_summary["% Passing Math"] = school_summary["% Passing Math"].map("{:.2f}%".format)
school_summary["% Passing Reading"] = school_summary["% Passing Reading"].map("{:.2f}%".format)
school_summary["% Overall Pass Rate"] = school_summary["% Overall Pass Rate"].map("{:.2f}%".format)


In [196]:
top_five = school_summary.sort_values("% Overall Pass Rate", ascending=False)
top_five.head(5)

,School Name,Type,Total Students,Budget,Per Student Budget,Avg Math,Avg Reading,% Passing Math,% Passing Reading,% Overall Pass Rate,Num Students
5,Wilson High School,Charter,2283,"$1,319,574",$578.0,83.27%,83.99%,90.93%,93.25%,92.09%,"2,283"
9,Pena High School,Charter,962,"$585,858",$609.0,83.84%,84.04%,91.68%,92.20%,91.94%,962
10,Wright High School,Charter,1800,"$1,049,400",$583.0,83.68%,83.95%,90.28%,93.44%,91.86%,"1,800"
6,Cabrera High School,Charter,1858,"$1,081,356",$582.0,83.06%,83.98%,89.56%,93.86%,91.71%,"1,858"
8,Holden High School,Charter,427,"$248,087",$581.0,83.80%,83.81%,90.63%,92.74%,91.69%,427


In [197]:
bottom_five = school_summary.sort_values("% Overall Pass Rate")
bottom_five.head(5)

,School Name,Type,Total Students,Budget,Per Student Budget,Avg Math,Avg Reading,% Passing Math,% Passing Reading,% Overall Pass Rate,Num Students
11,Rodriguez High School,District,3999,"$2,547,363",$637.0,76.84%,80.74%,64.07%,77.74%,70.91%,"3,999"
0,Huang High School,District,2917,"$1,910,635",$655.0,76.63%,81.18%,63.32%,78.81%,71.07%,"2,917"
12,Johnson High School,District,4761,"$3,094,650",$650.0,77.07%,80.97%,63.85%,78.28%,71.07%,"4,761"
1,Figueroa High School,District,2949,"$1,884,411",$639.0,76.71%,81.16%,63.75%,78.43%,71.09%,"2,949"
3,Hernandez High School,District,4635,"$3,022,020",$652.0,77.29%,80.93%,64.75%,78.19%,71.47%,"4,635"


In [198]:
ninth = student_data.loc[student_data["grade"] == "9th"].groupby("school_name", as_index=False)
tenth = student_data.loc[student_data["grade"] == "10th"].groupby("school_name", as_index=False)
eleventh = student_data.loc[student_data["grade"] == "11th"].groupby("school_name", as_index=False)
twelfth = student_data.loc[student_data["grade"] == "12th"].groupby("school_name", as_index=False)

In [199]:
ninth_math_avg = pd.DataFrame(ninth['math_score'].mean())
tenth_math_avg = pd.DataFrame(tenth['math_score'].mean())
eleventh_math_avg = pd.DataFrame(eleventh['math_score'].mean())
twelfth_math_avg = pd.DataFrame(twelfth['math_score'].mean())

In [200]:
math_by_grade = pd.merge(ninth_math_avg, tenth_math_avg, on="school_name")
math_by_grade = pd.merge(math_by_grade, eleventh_math_avg, on="school_name")
math_by_grade = pd.merge(math_by_grade, twelfth_math_avg, on="school_name")
math_by_grade.columns = ["School Name", "9th", "10th", "11th", "12th"]
math_by_grade.head(15)

,School Name,9th,10th,11th,12th
0,Bailey High School,77.083676,76.996772,77.515588,76.492218
1,Cabrera High School,83.094697,83.154506,82.765560,83.277487
2,Figueroa High School,76.403037,76.539974,76.884344,77.151369
3,Ford High School,77.361345,77.672316,76.918058,76.179963
4,Griffin High School,82.044010,84.229064,83.842105,83.356164
5,Hernandez High School,77.438495,77.337408,77.136029,77.186567
6,Holden High School,83.787402,83.429825,85.000000,82.855422
7,Huang High School,77.027251,75.908735,76.446602,77.225641
8,Johnson High School,77.187857,76.691117,77.491653,76.863248
9,Pena High School,83.625455,83.372000,84.328125,84.121547


In [201]:
ninth_reading_avg = pd.DataFrame(ninth['reading_score'].mean())
tenth_reading_avg = pd.DataFrame(tenth['reading_score'].mean())
eleventh_reading_avg = pd.DataFrame(eleventh['reading_score'].mean())
twelfth_reading_avg = pd.DataFrame(twelfth['reading_score'].mean())

In [202]:
reading_by_grade = pd.merge(ninth_reading_avg, tenth_reading_avg, on="school_name")
reading_by_grade = pd.merge(reading_by_grade, eleventh_reading_avg, on="school_name")
reading_by_grade = pd.merge(reading_by_grade, twelfth_reading_avg, on="school_name")
reading_by_grade.columns = ["School Name", "9th", "10th", "11th", "12th"]
reading_by_grade.head(15)

,School Name,9th,10th,11th,12th
0,Bailey High School,81.303155,80.907183,80.945643,80.912451
1,Cabrera High School,83.676136,84.253219,83.788382,84.287958
2,Figueroa High School,81.198598,81.408912,80.640339,81.384863
3,Ford High School,80.632653,81.262712,80.403642,80.662338
4,Griffin High School,83.369193,83.706897,84.288089,84.013699
5,Hernandez High School,80.866860,80.660147,81.396140,80.857143
6,Holden High School,83.677165,83.324561,83.815534,84.698795
7,Huang High School,81.290284,81.512386,81.417476,80.305983
8,Johnson High School,81.260714,80.773431,80.616027,81.227564
9,Pena High School,83.807273,83.612000,84.335938,84.591160


In [203]:
spending_bins = [0, 585, 615, 645, 675]
group_names = ["0 to 585", "585 to 615", "615 to 645", "645 to 675"]
score_by_spending = school_summary_og[["Avg Math", "Avg Reading", "% Passing Math", "% Passing Reading", "% Overall Pass Rate"]].groupby(pd.cut(school_summary_og["Per Student Budget"], bins=spending_bins, labels=group_names )).mean()
score_by_spending.head()

,Avg Math,Avg Reading,% Passing Math,% Passing Reading,% Overall Pass Rate
Per Student Budget,,,,,
0 to 585,83.455399,83.933814,90.350436,93.325838,91.838137
585 to 615,83.599686,83.885211,90.788049,92.410786,91.599418
615 to 645,79.079225,81.891436,73.021426,83.214343,78.117884
645 to 675,76.997210,81.027843,63.972368,78.427809,71.200088


In [204]:
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
score_by_size = school_summary_og[["Avg Math", "Avg Reading", "% Passing Math", "% Passing Reading", "% Overall Pass Rate"]].groupby(pd.cut(school_summary_og["Total Students"], bins=size_bins, labels=group_names )).mean()
score_by_size.head()

,Avg Math,Avg Reading,% Passing Math,% Passing Reading,% Overall Pass Rate
Total Students,,,,,
Small (<1000),83.821598,83.929843,91.158155,92.471895,91.815025
Medium (1000-2000),83.374684,83.864438,89.931303,93.244843,91.588073
Large (2000-5000),77.746417,81.344493,67.631335,80.190800,73.911067


In [205]:
school_summary_type = school_summary_og
school_summary_type["Type"] = school_summary_type["Type"].replace({"Charter": 1, "District":2})

bins = [0, 1, 2]
group_names = ["Charter", "District"]
scoreByType = school_summary_type[["Avg Math", "Avg Reading", "% Passing Math", "% Passing Reading", "% Overall Pass Rate"]].groupby(pd.cut(school_summary_type["Type"], bins=bins,labels=group_names)).mean()
scoreByType.head()

,Avg Math,Avg Reading,% Passing Math,% Passing Reading,% Overall Pass Rate
Type,,,,,
Charter,83.473852,83.896421,90.363226,93.052812,91.708019
District,76.956733,80.966636,64.302528,78.324559,71.313543
